In [117]:
import pandas as pd
import numpy as np
import random
import string

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_letters+string.digits
    rand_int = random.randint(1,4)
#     print(rand_int)
    string_random = [random.choice(letters) for i in range(stringLength)]
    chunk_array = []
    for chunk in chunks(string_random,rand_int):
        chunk_array.append("".join(chunk))
#     for i in range(rand_int+1):
    
#         string_random = string_random +" " + ''.join(random.choice(letters) for i in range(stringLength))
    
    return chunk_array

print(randomString())

['36Vv', 'FLhM', 'SZ']


In [118]:
print(randomString())

['l', 'q', 'A', 'l', 'x', 's', 'z', 'A', '3', '7']


In [119]:
data = open("dummyData.txt").readlines()

In [120]:
import spacy

nlp = spacy.load("en_core_web_sm")

words_list, embedding_matrix = utils.load_glove()
unword_n = len(words_list)


In [121]:
array = []


for textindex,text in enumerate(data):
    doc = nlp(text.strip())
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #        token.shape_, token.is_alpha, token.is_stop)
        text_items = [token.text]
        if token.text == "XYYZ":
            tag = "Destination"
            rand_int = random.randint(1,4)
            if rand_int==3:
                text_items = ["Unknown"]
            else:
                text_items = randomString()
        else:
            tag = ""
            rand_int = random.randint(1,10)
            if rand_int==3:
                text_items = ["Unknown"]
        for item in text_items:
            array.append(dict(Sentence=textindex,Word=item,POS=token.pos_,Tag=tag))

In [122]:
df = pd.DataFrame(array)  

In [123]:
df.tail(10)

,POS,Sentence,Tag,Word
1091,VERB,122,,got
1092,PART,122,,to
1093,AUX,122,,get
1094,NOUN,122,,directions
1095,ADP,122,,to
1096,PROPN,122,Destination,Jb
1097,PROPN,122,Destination,0v
1098,PROPN,122,Destination,R5
1099,PROPN,122,Destination,fB
1100,PROPN,122,Destination,sw


In [124]:
words = list(set(df["Word"].values))
words.append("ENDPAD")
words.append("Unknown")
n_words = len(words); n_words

283

In [125]:
tags = list(set(df["Tag"].values))
n_tags = len(tags); n_tags

pos = list(set(df["POS"].values))
pos.append("BLANK")
pos_tags = len(pos)
print(n_tags,pos_tags)

2 13


In [126]:
agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                   s["POS"].values.tolist(),
                                                   s["Tag"].values.tolist())]
grouped = df.groupby("Sentence").apply(agg_func)

In [127]:
sentences = [s for s in grouped]

In [128]:
print(sentences[0])

[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Unknown', 'PROPN', 'Destination')]


In [129]:
def yield_sentence():
    for group in grouped:
        yield group
    yield None

In [130]:
for group in yield_sentence():
    print(group)
    print("-------")

[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Unknown', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('Unknown', 'ADP', ''), ('5y', 'PROPN', 'Destination'), ('xW', 'PROPN', 'Destination'), ('XK', 'PROPN', 'Destination'), ('2l', 'PROPN', 'Destination'), ('xW', 'PROPN', 'Destination'), ('home', 'ADV', '')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('L', 'PROPN', 'Destination'), ('l', 'PROPN', 'Destination'), ('s', 'PROPN', 'Destination'), ('W', 'PROPN', 'Destination'), ('d', 'PROPN', 'Destination'), ('w', 'PROPN', 'Destination'), ('o', 'PROPN', 'Destination'), ('5', 'PROPN', 'Destination'), ('h', 'PROPN', 'Destination'), ('N', 'PROPN', 'Destination'), ('work', 'NOUN', '')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('the', 'DET', ''), ('closest', 'ADJ', ''), ('ouVM', 'PROPN', 'Destination'), ('nXel', 'PROPN', 'Destination'), ('xy', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Unknown', 'DET', ''), ('nea

In [131]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
pos2idx = {p: i for i, p in enumerate(pos)}


In [132]:
word2idx["directions"]

12

In [133]:
tag2idx[""]
pos2idx

{'VERB': 0,
 'PUNCT': 1,
 'ADV': 2,
 'AUX': 3,
 'NUM': 4,
 'DET': 5,
 'ADP': 6,
 'ADJ': 7,
 'PRON': 8,
 'PART': 9,
 'PROPN': 10,
 'NOUN': 11,
 'BLANK': 12}

In [134]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx.get(w[0],word2idx["Unknown"]) for w in s] for s in sentences]
X_pos = [[pos2idx.get(w[1]) for w in s] for s in sentences]

In [135]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)
X_pos = pad_sequences(maxlen=max_len, sequences=X_pos, padding="post", value=pos2idx["BLANK"])
# print(X.shape)
# print(X_pos.shape)

# X = np.stack((X,X_pos))
# print(X.shape)
# X = np.reshape(X,[123,2,75])
# print(X.shape)

In [136]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]

In [137]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx[""])


In [138]:
from keras.utils import to_categorical

In [139]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [140]:
from sklearn.model_selection import train_test_split

In [141]:
X_tr, X_te, y_tr, y_te, X_pos_tr, X_pos_te = train_test_split(X, y, X_pos, test_size=0.1)

In [1]:
import pickle
import pandas as pd
import numpy as np
import random
import string
pickle_in = open("training_and_testing_data.pickle","rb")

data_dict = pickle.load(pickle_in)

In [5]:

X_tr = data_dict["X_tr"]
X_te = data_dict["X_te"]
y_tr = data_dict["y_tr"]
y_te = data_dict["y_te"]
X_pos_tr = data_dict["X_pos_tr"]
X_pos_te = data_dict["X_pos_te"]

max_len = 75
n_words = data_dict["n_words"]
n_tags = data_dict["n_tags"]
tag2idx = data_dict["tag2idx"]
pos2idx = data_dict["pos2idx"]
word2idx = data_dict["word2idx"]
pos = data_dict["pos"]

In [6]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
import keras

In [7]:
# input = Input(shape=(max_len,))
# model = Embedding(input_dim=n_words + 1, output_dim=20,
#                   input_length=max_len)(input)  # 20-dim embedding
# model = Bidirectional(LSTM(units=50, return_sequences=True,
#                            recurrent_dropout=0.1))(model)  # variational biLSTM
# model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
# crf = CRF(n_tags)  # CRF layer
# out = crf(model)  # output


big_input = Input(shape=(150,))
input = Input(shape=(max_len,))
word_emb = Embedding(input_dim=n_words + 1, output_dim=20,
input_length=max_len)(input) # 20-dim embedding

pos_input = Input(shape=(max_len,))
pos_emb = Embedding(input_dim= len(pos) , output_dim=10,
input_length=max_len)(pos_input)

modified_input = keras.layers.concatenate([word_emb, pos_emb])

model_1 = Bidirectional(LSTM(units=50, return_sequences=True,
recurrent_dropout=0.1))(modified_input)

model = TimeDistributed(Dense(50, activation="relu"))(model_1)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output



In [8]:
model = Model([input,pos_input], out)

In [9]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

/home/premraj/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/premraj/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [10]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 75)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 75)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 75, 20)       1620        input_5[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 75, 10)       140         input_6[0][0]                    
____________________________________________________________________________________________

In [11]:
print(X_tr[0].shape)

(75,)


In [13]:
history = model.fit([X_tr,X_pos_tr], np.array(y_tr), batch_size=32, epochs=60,
                    validation_split=0.1, verbose=1)

Train on 137 samples, validate on 16 samples
Epoch 1/60
137/137 [==============================] - 0s 3ms/step - loss: 0.0132 - crf_viterbi_accuracy: 0.9982 - val_loss: 0.0122 - val_crf_viterbi_accuracy: 0.9983
Epoch 2/60
137/137 [==============================] - 0s 3ms/step - loss: 0.0116 - crf_viterbi_accuracy: 0.9980 - val_loss: 0.0177 - val_crf_viterbi_accuracy: 0.9950
Epoch 3/60
137/137 [==============================] - 0s 3ms/step - loss: 0.0115 - crf_viterbi_accuracy: 0.9986 - val_loss: 0.0107 - val_crf_viterbi_accuracy: 0.9983
Epoch 4/60
137/137 [==============================] - 0s 3ms/step - loss: 0.0097 - crf_viterbi_accuracy: 0.9992 - val_loss: 0.0102 - val_crf_viterbi_accuracy: 0.9983
Epoch 5/60
137/137 [==============================] - 0s 3ms/step - loss: 0.0084 - crf_viterbi_accuracy: 0.9995 - val_loss: 0.0098 - val_crf_viterbi_accuracy: 0.9992
Epoch 6/60
137/137 [==============================] - 0s 3ms/step - loss: 0.0079 - crf_viterbi_accuracy: 0.9993 - val_loss: 0

Epoch 50/60
137/137 [==============================] - 0s 3ms/step - loss: 0.0024 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0054 - val_crf_viterbi_accuracy: 0.9992
Epoch 51/60
137/137 [==============================] - 0s 3ms/step - loss: 0.0023 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0063 - val_crf_viterbi_accuracy: 0.9992
Epoch 52/60
137/137 [==============================] - 0s 4ms/step - loss: 0.0022 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0062 - val_crf_viterbi_accuracy: 0.9992
Epoch 53/60
137/137 [==============================] - 0s 4ms/step - loss: 0.0022 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0060 - val_crf_viterbi_accuracy: 0.9992
Epoch 54/60
137/137 [==============================] - 0s 3ms/step - loss: 0.0021 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0062 - val_crf_viterbi_accuracy: 0.9992
Epoch 55/60
137/137 [==============================] - 0s 3ms/step - loss: 0.0020 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0060 - val_crf_viterbi_accuracy: 0.999

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


In [15]:
test_pred = model.predict([X_te,X_pos_te], verbose=1)

18/18 [==============================] - 0s 16ms/step


In [16]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", ""))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [17]:
def getRecallPrecision(test_labels,pred_labels,tag):
    flat_test_labels = [item for sublist in test_labels for item in sublist]
    flat_pred_labels = [item for sublist in pred_labels for item in sublist]
    print(flat_test_labels)
    print(flat_pred_labels)
    test_indices = set([i for i, x in enumerate(flat_test_labels) if x == tag])
    pred_indices = set([i for i, x in enumerate(flat_pred_labels) if x == tag])
    recall = len(test_indices.intersection(pred_indices))/len(test_indices)
    precision = len(test_indices.intersection(pred_indices))/len(pred_indices)
    f_score = 2* precision* recall/(precision + recall)
    return recall,precision,f_score

In [19]:
print(test_labels)
print("-----")
print(pred_labels)
print(getRecallPrecision(test_labels,pred_labels,"Destination"))

[['', '', '', '', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', 'Destination', '', '', 'Destination', 'Destination', ''

In [22]:
import spacy
import pandas as pd
import numpy as np
import random
import string
import spacy
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pickle
nlp = spacy.load("en_core_web_sm")

In [36]:
sentence = "Where do you want to go? Never mind"
entities_words = ["Neve"]
array = []
doc = nlp(sentence)
for token in doc:
    text = token.text
    print(text)
    if token.text in entities_words:
        tag = "Destination"
    else:
        tag = ""

    array.append(tuple([token.text,token.pos_,tag]))
print(array)
X = [[word2idx.get(w[0],word2idx["Unknown"]) for w in s] for s in [array]]
X_pos = [[pos2idx.get(w[1]) for w in s] for s in [array]]

print(X)
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)
X_pos = pad_sequences(maxlen=max_len, sequences=X_pos, padding="post", value=pos2idx["BLANK"])
y = [[tag2idx[w[2]] for w in s] for s in [array]]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx[""])
y = [to_categorical(i, num_classes=n_tags) for i in y]
#print(y)
test_pred = model.predict([X,X_pos], verbose=1)
pred_labels = pred2label(test_pred)
test_labels = pred2label(y)
print(test_labels,pred_labels)
#print(pred_labels)

Where
do
you
want
to
go
?
Never
mind
[('Where', 'ADV', ''), ('do', 'AUX', ''), ('you', 'PRON', ''), ('want', 'VERB', ''), ('to', 'PART', ''), ('go', 'VERB', ''), ('?', 'PUNCT', ''), ('Never', 'ADV', ''), ('mind', 'VERB', '')]
[[13, 7, 38, 55, 18, 14, 73, 80, 80]]
1/1 [==============================] - 0s 22ms/step
[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']] [['', '', '', '', '', '', '', 'Destination', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]


In [ ]:
print(word2idx)